# Purpose

This notebook will find the 50k most active Lichess players (rapid blitz classical), measured by number of games.

It does this by downloading and reading parquet files of raw game data.

It's not practical to download the hundreds of 1GB parquet files; so we will download a certain number and extrapolate from there.

We don't need to know exactly who is the *most* active, just need to find *very* active players.

Then, we will process their games for data to feed in to our chess opening recommender AI model.

# Reason

Originally, we 